# SFC版DQ3の乱数生成器

## TL;DR
#### <b>SFC版DQ3の乱数は割と質が良さそう！</b>

## 1. 乱数生成器の概要
SFC版DQ3の乱数生成器は下記のようなアルゴリズムになっています。

```
  rand() -> uint8 := R & 0xff

where, R: uint32 := The state of a Random Number Generator, transited by following
    R(0) = 0xaae21259
    R(n) = (R(n-1) * 256) ^ (((R(n-1).range(0, 16) << 2) ^ R(n-1).range(16, 32)) << 1) >> 8
  
    n.range(lower, upper) := n & (2^upper - 1) >> lower
```

乱数生成器は下記の2通りの場合に遷移が発生します。

1. 乱数の消費が行われたとき (SR at 0012e3)
2. フレームが進んだとき (SR at 0006a1)

ということで、実際の値を見ていきましょう。

In [2]:
// Import the library
:dep dq3 = { path = "../" }
use dq3::rand::*;

In [3]:
// Construct a RNG and print the initial state.
let mut rng = Generator::new(None);
println!("R(0): {}", rng.state());

R(0): 0xaae21259


Rの初期値はこんな感じです。

続いて、何回か乱数生成器を遷移させて、乱数を取得してみましょう。

In [4]:
// Get rands 4 times.
let iter = 5;
for i in 1..iter {
    println!("rand({}): {}", i, rng.rand());
    println!("R({}): {}", i, rng.state());   
}

rand(1): 199
R(1): 0xe21259c7
rand(2): 10
R(2): 0x1259c70a
rand(3): 28
R(3): 0x59c70a1c
rand(4): 227
R(4): 0xc70a1ce3


()

## 2. 派生された乱数取得処理
### 2.1. 乗算と除算による [0, upper] の一様乱数取得

上記のrand関数は [0, 255] の一様整数乱数を返します。<br>
しかし実際には、指定した範囲の乱数値を取得したい場合が多々あります。

この場合は、 `SR:00133e` によって下記の計算式で `[0, upper]` の一様整数乱数を計算しています。

```
rand_by_multiply(upper: uint8) -> uint8 := uint8(uint16(rand()) * (upper + 1) / 256)
```

このサブルーチンは、例えばエンカウント歩数の初期値計算([0, 16]の一様整数乱数を取得)などで利用されています。

In [5]:
// Get a rand inbound [0, 16].
println!("{}", rng.rand_by_multiply(16));
println!("{}", rng.state());

6
0xa1ce369


## 2.2. 多項分布に従った乱数取得

一様分布ではなくて、正規分布に近いような乱数を取得したい場合もあるでしょう。<br>
この場合は、 `SR: 0014d4` によって、 下記の計算によって多項分布に従った乱数を取得します。

```
  rand_multinomial(offset, mask) = (offset + multi(n=16, p= rand() & mask)) % 256
```

わかりやすく説明すると、下記のようになります。

  1. rand() によって取得した [0, 255] の一様整数乱数と mask の論理積を取り、 [0, mask]の一様整数乱数に変換する。(mask はおそらく 2^n-1 に当てはまる整数値を前提としている。)
  2. 2　を16回施行した結果と offset を足し合わせ、 256で割った値を結果とする。
  
  
このサブルーチンは、例えば下記の値決定の乱数に利用されています。

* 物理ダメージ計算: (offset=6, mask=0xf)
* 行動順値計算: (offset=136, mask=0x1f)
* ステータス成長値計算: (offset=136, mask=0x1f)

In [6]:
// Get a rand of actiion order value (or a rise of a states)
println!("{}", rng.rand_multinomial(136, 0x1f));

192


# 3. 乱数の特性を調べてみる
## 3.1. RNGの状態値の周期性

以上で基本は抑えられたので、ここからは乱数の様々な特性を見ていきましょう。<br>

まずは基本となるRNGの周期性を調べてみます。

In [21]:
use std::collections::HashSet;

// Found interval of RNG.
fn rng_sequence_in_interval() -> Vec<State> {
    let mut rng = Generator::new(None);
    let interval_max: usize = 0x100000000;
    let mut sequence = Vec::new();
    let mut generated = HashSet::new();
    
    for _ in 0..interval_max {
        rng.transit();
        
        let state = rng.state();    
        if generated.contains(&state) {
            break;
        }
        
        sequence.push(state);
        generated.insert(state);
    }
    
    let begin = sequence.iter().position(|v| *v == rng.state()).unwrap();
    
    println!("Loopback to index: {:x}", begin);
    
    let mut ret = Vec::new();
    ret.extend_from_slice(&sequence[begin..]);
    return ret;
}

let seq = rng_sequence_in_interval();
println!("Interval: {:x}", seq.len());

Loopback to index: 0
Interval: 7fffffff


RNGの周期は `0x7ffffff` (= `2^31-1`) になるようです。1周した後再度初期値に戻っているので、1度のみしか算出されない値もないようですね。

## 3.2. rand 関数の出現頻度

次は1周期中における `rand` 関数が返す8-bit乱数のヒストグラムを生成してみましょう。

In [22]:
:dep ndarray = "0.13.1"
use ndarray;

let mut histogram: ndarray::Array1<usize> = ndarray::Array::zeros(256);
for state in &seq {
    histogram[state.rand() as usize] += 1;
}
println!("{}", histogram);

[8388607, 8388608, 8388608, 8388608, 8388608, 8388608, 8388608, 8388608, 8388608, 8388608, 8388608, 8388608, 8388608, 8388608, 8388608, 8388608, 8388608, 8388608, 8388608, 8388608, 8388608, 8388608, 8388608, 8388608, 8388608, 8388608, 8388608, 8388608, 8388608, 8388608, 8388608, 8388608, 8388608, 8388608, 8388608, 8388608, 8388608, 8388608, 8388608, 8388608, 8388608, 8388608, 8388608, 8388608, 8388608, 8388608, 8388608, 8388608, 8388608, 8388608, 8388608, 8388608, 8388608, 8388608, 8388608, 8388608, 8388608, 8388608, 8388608, 8388608, 8388608, 8388608, 8388608, 8388608, 8388608, 8388608, 8388608, 8388608, 8388608, 8388608, 8388608, 8388608, 8388608, 8388608, 8388608, 8388608, 8388608, 8388608, 8388608, 8388608, 8388608, 8388608, 8388608, 8388608, 8388608, 8388608, 8388608, 8388608, 8388608, 8388608, 8388608, 8388608, 8388608, 8388608, 8388608, 8388608, 8388608, 8388608, 8388608, 8388608, 8388608, 8388608, 8388608, 8388608, 8388608, 8388608, 8388608, 8388608, 8388608, 8388608, 8388608, 

0のみ `2^23-1` 通り、 他の値は `2^23` 通り生成されるようです。一様な整数乱数が生成出来ていると言って良いでしょう。

### 3.3. 連続する2次元の乱数の出現頻度

今度は、1周期中における `rand` 関数を2連続で取得した場合のヒストグラムを生成してみます。<br>
疑似乱数のアルゴリズムによっては多次元の乱数を取得すると偏りが出てくる場合があります。<br>
つまり、次に生成される乱数が前に生成された乱数に依存してしまうということです。
この顕著な的な例が [SFCDQ2におけるはぐれメタルのHP決定](http://maru0137.hatenablog.com/entry/2016/10/04/190946) ですね。<br>
一方で本当にランダムであれば、以前に生成された乱数からは全く依存を受けないはずで、この方が良い疑似乱数アルゴリズムとされています。<br>
(1998年に `Mersenne Twister` というとても質の良い疑似乱数生成アルゴリズムが発表されており、現在ではこちらが広く使われています。PS2DQ5もこれを使っているのが確認されています。)<br>
(ちなみに[PS版DQ4/DQ7では線形合同法を利用して乱数を生成しています](https://ch.nicovideo.jp/maru0137/blomaga/ar638062)。ちなみにこれは、[一昔前のglibcのrand関数](https://github.com/bminor/glibc/blob/master/stdlib/random_r.c#L364)とアルゴリズムもパラメータも全く同じ。)


ということで、乱数値の2次元のヒストグラムを生成してみましょう。

In [23]:
let mut hist2d: ndarray::Array2<usize> = ndarray::Array::zeros((256, 256));

for i0 in 0..seq.len() {
    let i1 = (i0 + 1) % seq.len();
    let v0 = seq[i0].rand();
    let v1 = seq[i1].rand();
    hist2d[[v0 as usize, v1 as usize]] +=1;
}

println!("{}", hist2d);

[[32767, 32768, 32768, 32768, 32768, ..., 32768, 32768, 32768, 32768, 32768],
 [32768, 32768, 32768, 32768, 32768, ..., 32768, 32768, 32768, 32768, 32768],
 [32768, 32768, 32768, 32768, 32768, ..., 32768, 32768, 32768, 32768, 32768],
 [32768, 32768, 32768, 32768, 32768, ..., 32768, 32768, 32768, 32768, 32768],
 [32768, 32768, 32768, 32768, 32768, ..., 32768, 32768, 32768, 32768, 32768],
 ...,
 [32768, 32768, 32768, 32768, 32768, ..., 32768, 32768, 32768, 32768, 32768],
 [32768, 32768, 32768, 32768, 32768, ..., 32768, 32768, 32768, 32768, 32768],
 [32768, 32768, 32768, 32768, 32768, ..., 32768, 32768, 32768, 32768, 32768],
 [32768, 32768, 32768, 32768, 32768, ..., 32768, 32768, 32768, 32768, 32768],
 [32768, 32768, 32768, 32768, 32768, ..., 32768, 32768, 32768, 32768, 32768]]


値が `(0. 0)` の場合だけ `2^15-1` 通り、 他は `2^15` 通りで、ほぼ一様となっています。<br>
おっ、これはかなり質の良い方の乱数だと言えるのではないでしょうか！

## 4. ゲーム内で実際に利用されている乱数の分布を求めてみる
### 4.1. エンカウント歩数の初期値

まずはヒストグラムから生成してみます。

In [24]:
let upper_in_trial: u8 = 16;
let trial: usize = 3;
let bin_num: usize = (upper_in_trial as usize) * trial + 1;
let offset: usize = 0;

let mut enc_step_seq = Vec::new();

for state in &seq {    
    let mut rng = Generator::new(Some(*state));
    
    let mut sum: u8 = offset as u8;
    for _ in 0..trial {
        sum += rng.rand_by_multiply(upper_in_trial);
    }
    
    enc_step_seq.push(sum);
}

let mut enc_step_hists: ndarray::Array1<usize> = ndarray::Array::zeros(bin_num);
for enc_step in &enc_step_seq {        
    enc_step_hists[*enc_step as usize] += 1;
}


println!("{}", enc_step_hists);

[524287, 1474560, 2856960, 4671360, 6917760, 9596160, 12706560, 16248960, 20223360, 24629760, 29468160, 34738560, 40440960, 46575360, 53141760, 60140160, 67570560, 73958400, 79488000, 84153600, 87955200, 90892800, 92966400, 94176000, 94521600, 94003200, 92620800, 90374400, 87264000, 83289600, 78451200, 72748800, 66182400, 58752000, 51840000, 45360000, 39312000, 33696000, 28512000, 23760000, 19440000, 15552000, 12096000, 9072000, 6480000, 4320000, 2592000, 1296000, 432000]


これを使って分布を求めてみましょう。単位は％です。

In [25]:
let mut enc_step_dists: ndarray::Array1<f64> = ndarray::Array::zeros(bin_num);

for i in 0..enc_step_hists.len() {
    enc_step_dists[i] = enc_step_hists[i] as f64 * 100_f64 / (seq.len() as f64); 
}
println!("{}", enc_step_dists);

[0.024414015945239932, 0.06866455081322442, 0.13303756720062232, 0.21752715120908206, 0.32213330283860364, 0.44685602208918707, 0.5916953089608323, 0.7566511634535394, 0.9417235855673084, 1.1469125753021392, 1.3722181326580318, 1.6176402576349864, 1.8831789502330027, 2.168834210452081, 2.474606038292221, 2.8004944337534226, 3.1464993968356865, 3.4439563767257875, 3.701448442275379, 3.918707372582847, 4.095733167648191, 4.232525827471412, 4.329085352052509, 4.385411741391482, 4.401504995488331, 4.377365114343057, 4.312992097955659, 4.208385946326137, 4.0635466594544924, 3.8784742373407233, 3.6531686799848306, 3.3876299873868145, 3.0818581595466745, 2.7358531964644106, 2.413988114527421, 2.1122396002114936, 1.8306076535166278, 1.5690922744428237, 1.3276934629900816, 1.1064112191584015, 0.9052455429477829, 0.7241964343582263, 0.5632638933897316, 0.4224479200422987, 0.30174851431592764, 0.20116567621061843, 0.12069940572637106, 0.06034970286318553, 0.020116567621061843]


[乱数が独立であることを前提にした場合の確率表](http://bamboo.ninja-web.net/dq3/DQ3encsteps_init/DQ3encsteps_init.html)と比べてみると、ほとんど差が無いことが分かります。

グラフも描画してみます。

In [26]:
:dep plotters = { git = "https://github.com/38/plotters", default_features = false, features = ["evcxr", "histogram"] }
use plotters;
use plotters::prelude::*;

let figure = evcxr_figure((1920, 1080), |root| {
    root.fill(&WHITE);

    let mut chart = ChartBuilder::on(&root)
        .x_label_area_size(60)
        .y_label_area_size(40)
        .margin(5)
        .caption("Distribution of Encount Step", ("sans-serif", 30.0).into_font())
        .build_ranged(0u32..bin_num as u32, 0_f64..5_f64)?;

    chart
        .configure_mesh()
        .y_label_formatter(&|y| format!("{}%", *y))
        .disable_x_mesh()
        .disable_y_mesh()
        .x_label_offset(30)
        .y_desc("Probability")
        .x_desc("Encount Step")
        .axis_desc_style(("sans-serif", 15).into_font())
        .draw()?;
    
    chart.draw_series(
        Histogram::vertical(&chart)
            .style(RED.mix(0.5).filled())
            .data(enc_step_seq.iter().map(|x: &u8| (*x as u32, 100_f64 / enc_step_seq.len() as f64))),
    )?;
    
    Ok(())
});
figure

Distribution of Encount Step
 
 
Probability
 
 
Encount Step
 
 
 
0.5%
 
 
 
1%
 
 
 
1.5%
 
 
 
2%
 
 
 
2.5%
 
 
 
3%
 
 
 
3.5%
 
 
 
4%
 
 
 
4.5%
 
 
 
 
0
 
 
 
10
 
 
 
20
 
 
 
30
 
 
 
40

### 4.2. 物理ダメージ計算式における乱数分布

まずは `[99, 153]` にclampする前の値を求めます。
    

In [27]:
let offset: u8 = 6_u8;
let mask: u8 = 0xf_u8;
let bin_num: usize = 256;

let mut damage_rand_seq = Vec::new();
for state in &seq {    
    let mut rng = Generator::new(Some(*state));    
    damage_rand_seq.push(rng.rand_multinomial(offset, mask));
}

()

In [28]:
let figure = evcxr_figure((3840, 2160), |root| {
    root.fill(&WHITE);

    let mut chart = ChartBuilder::on(&root)
        .x_label_area_size(35)
        .y_label_area_size(40)
        .margin(5)
        .caption("Distribution of Damage Rand(Before clamped)", ("sans-serif", 30.0).into_font())
        .build_ranged(0_u32..bin_num as u32, 0_f64..2.5_f64)?;

    chart
        .configure_mesh()
        .y_label_formatter(&|y| format!("{}%", *y))
        .disable_x_mesh()
        .disable_y_mesh()
        .x_labels(32)
        .x_label_offset(30)
        .y_desc("Probability")
        .x_desc("Physical Damage Rand (Before clamp)")
        .axis_desc_style(("sans-serif", 15).into_font())
        .draw()?;
    
    chart.draw_series(
        Histogram::vertical(&chart)
            .style(RED.mix(0.5).filled())
            .data(damage_rand_seq.iter().map(|x: &u8| (*x as u32, 100_f64 / damage_rand_seq.len() as f64))),
    )?;
    
    Ok(())
});
figure

Distribution of Damage Rand(Before clamped)
 
 
Probability
 
 
Physical Damage Rand (Before clamp)
 
 
 
0.5%
 
 
 
1%
 
 
 
1.5%
 
 
 
2%
 
 
 
 
0
 
 
 
10
 
 
 
20
 
 
 
30
 
 
 
40
 
 
 
50
 
 
 
60
 
 
 
70
 
 
 
80
 
 
 
90
 
 
 
100
 
 
 
110
 
 
 
120
 
 
 
130
 
 
 
140
 
 
 
150
 
 
 
160
 
 
 
170
 
 
 
180
 
 
 
190
 
 
 
200
 
 
 
210
 
 
 
220
 
 
 
230
 
 
 
240
 
 
 
250

実際の物理ダメージ計算ではこの後、 `[99, 153]` にclampされて利用されます。
ということでclampしてみましょう。

In [29]:
let mut damage_rand_clamped_seq = Vec::new();
for rand in &damage_rand_seq {    
    damage_rand_clamped_seq.push(std::cmp::max(99_u8, std::cmp::min(153_u8, *rand)));
}

()

In [30]:
let figure = evcxr_figure((1920, 1080), |root| {
    root.fill(&WHITE);

    let mut chart = ChartBuilder::on(&root)
        .x_label_area_size(60)
        .y_label_area_size(40)
        .margin(5)
        .caption("Distribution of Damage Rand", ("sans-serif", 30.0).into_font())
        .build_ranged(90_u32..160_u32, 0_f64..8_f64)?;

    chart
        .configure_mesh()
        .y_label_formatter(&|y| format!("{}%", *y))
        .disable_x_mesh()
        .disable_y_mesh()
        .x_label_offset(30)
        .y_desc("Probability")
        .x_desc("Physical Damage Rand")
        .axis_desc_style(("sans-serif", 15).into_font())
        .draw()?;
    
    chart.draw_series(
        Histogram::vertical(&chart)
            .style(RED.mix(0.5).filled())
            .data(damage_rand_clamped_seq.iter().map(|x: &u8| (*x as u32, 100_f64 / damage_rand_clamped_seq.len() as f64))),
    )?;
    
    Ok(())
});
figure

Distribution of Damage Rand
 
 
Probability
 
 
Physical Damage Rand
 
 
 
1%
 
 
 
2%
 
 
 
3%
 
 
 
4%
 
 
 
5%
 
 
 
6%
 
 
 
7%
 
 
 
 
90
 
 
 
100
 
 
 
110
 
 
 
120
 
 
 
130
 
 
 
140
 
 
 
150
 
 
 
160

### 4.3. 行動値計算 または ステータス成長値計算における乱数

In [31]:
let offset: u8 = 136_u8;
let mask: u8 = 0x1f_u8;
let bin_num: usize = 256;

let mut action_rand_seq = Vec::new();
for state in &seq {    
    let mut rng = Generator::new(Some(*state));    
    action_rand_seq.push(rng.rand_multinomial(offset, mask));
}

()

In [32]:
let figure = evcxr_figure((3840, 2160), |root| {
    root.fill(&WHITE);

    let mut chart = ChartBuilder::on(&root)
        .x_label_area_size(60)
        .y_label_area_size(40)
        .margin(5)
        .caption("Distribution of Action Order Rand", ("sans-serif", 30.0).into_font())
        .build_ranged(0_u32..256_u32 as u32, 0f64..1.2_f64)?;

    chart
        .configure_mesh()
        .y_label_formatter(&|y| format!("{}%", *y))
        .disable_x_mesh()
        .disable_y_mesh()
        .x_label_offset(30)
        .y_desc("Probability")
        .x_desc("Action Order Rand")
        .axis_desc_style(("sans-serif", 15).into_font())
        .draw()?;
    
    chart.draw_series(
        Histogram::vertical(&chart)
            .style(RED.mix(0.5).filled())
            .data(action_rand_seq.iter().map(|x: &u8| (*x as u32, 100_f64 / damage_rand_seq.len() as f64))),
    )?;
    
    Ok(())
});
figure

Distribution of Action Order Rand
 
 
Probability
 
 
Action Order Rand
 
 
 
0.2%
 
 
 
0.4%
 
 
 
0.6%
 
 
 
0.8%
 
 
 
1%
 
 
 
 
0
 
 
 
50
 
 
 
100
 
 
 
150
 
 
 
200
 
 
 
250

# 5. 参考文献
* [初期歩数の説明](http://bamboo.ninja-web.net/dq3/DQ3encsteps_init/DQ3encsteps_init.html)
* [SFC版DQ3 打撃ダメージ値乱数を考える](https://ch.nicovideo.jp/mugensai/blomaga/ar149138)
* [SFC版DQ3直接攻撃](http://baa.game-ss.com/dq3/dq3dageki)
* [SFC版ドラゴンクエスト３の行動順と素早さの関係](http://baa.game-ss.com/dq3/sfc%E7%89%88%E3%83%89%E3%83%A9%E3%82%B4%E3%83%B3%E3%82%AF%E3%82%A8%E3%82%B9%E3%83%88%EF%BC%93%E3%81%AE%E8%A1%8C%E5%8B%95%E9%A0%86%E3%81%A8%E7%B4%A0%E6%97%A9%E3%81%95%E3%81%AE%E9%96%A2%E4%BF%82)
